In [1]:
from transformers import AutoModel, AutoTokenizer, PreTrainedTokenizerFast, PreTrainedModel, BertTokenizer
model = "pritamdeka/S-PubMedBert-MS-MARCO"

In [2]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(model)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [5]:
from datasets import Dataset
ds = Dataset.from_json("/home/jovyan/output-cardiology/pubmed/data.jsonl")

Found cached dataset json (/home/jovyan/.cache/huggingface/datasets/json/default-c7ebc142947617f3/0.0.0)


In [101]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.text_splitter import NLTKTextSplitter


def split(batch: Dataset):
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer,
        chunk_size = 256,
        chunk_overlap = 20,
        separators=["\n\n", "\n", ".", " "]
    )
    new_rows = []
    text_input = []
    meta = []
    for idx, text in enumerate(batch['text']):
        text_input.append(text)
        meta.append({'index': idx})
    docs = text_splitter.create_documents(text_input, meta)
    columns = batch.column_names
    
    
    for doc in docs:
        new_row = {}
        new_row['text'] = doc.page_content
        old_row = batch[doc.metadata.index]
        new_row.update(old_row)
        new_rows.append(new_row)
    return new_rows    

In [103]:
ds2 = ds.map(split, batch_size=100, batched=True)

Map:   0%|          | 0/549563 [00:00<?, ? examples/s]

TypeError: string indices must be integers

In [97]:
docs = text_splitter.create_documents([x['text'] for x in all_data[:10]])

In [98]:
docs

[Document(page_content='The importance of the contribution of atrial systole to ventricular filling in mitral stenosis is controversial. The cause of reduced cardiac output following the onset of atrial fibrillation may be due to an increased heart rate, a loss of booster pump function, or both.', metadata={}),
 Document(page_content='Imaging of the flow convergence region (FCR) proximal to a regurgitant orifice has been shown to provide a method for quantifying the regurgitant flow rate. According to the continuity principle, the FCR is constituted by concentric hemispheric isovelocity surfaces centered at the orifice. The flow rate is constant across all isovelocity surfaces and equals the flow rate through the orifice. For any isovelocity surface the flow rate (Q) is given by: Q = 2 pi r2 Vr, where 2 pi r2 is the area of the hemisphere and Vr is the velocity at the radial distance (r) from the orifice.', metadata={}),
 Document(page_content='This study was designed to evaluate the e

In [84]:
idx = 110302
print('MAIN:')
print(all_data[idx]['text'])
docs = text_splitter.create_documents([all_data[idx]['text']])
print(f'SPLITS:{len(docs)}')

for doc in docs:
    print(len(doc.page_content),"**" + doc.page_content)


MAIN:
Part 1: Technical and methodological issues. Contrast echocardiography is based on the use of gas microbubbles. The size, gas composition and shell structure of the microbubbles modify their stability, resistance to pressure and scattering behavior. A proposed classification of contrast agents is based on the modalities of production of microbubbles (galenic or industrial); the industrial agents are divided into three generations depending on their characteristics. Following venous administration, the industrial microbubbles behave as intravascular free-flowing tracers and this is fundamental for their use in perfusion studies. When insonated at a low acoustic pressure, microbubbles show a linear behavior and can be used for signal amplification. At intermediate acoustic pressures microbubbles resonate and produce a harmonic signal that is detectable by new scanners. Higher acoustic pressures cause microbubble disruption with emission of a transient acoustic signal. The available

In [72]:
tokenizer([all_data[idx]['text']], padding=False, return_offsets_mapping=True, return_length=True)['length']

[1263]

In [75]:
for d in docs:
    print(tokenizer(d.page_content, padding=False, return_offsets_mapping=True, return_length=True)['length'])

[106]
[104]
[102]
[103]
[96]
[109]
[96]
[78]
[87]
[107]
[99]
[115]
[95]
[60]


In [92]:
from datasets.table import Table
import pyarrow as pa


In [96]:
pa.Table.from_pydict({'a': [1,2,3], 'b':['sdf', 'sdf', 'sdfsdfsdfsdfsdfdsf']})

Dataset(

SyntaxError: closing parenthesis ')' does not match opening parenthesis '{' (788907221.py, line 1)